# Data Cleaning
The prior data quality analysis revealed several data anomalies requiring attention. 

1. Cases with invalid values for subtlety and breast density.
2. Missing calcification type, calcification distribution, mass shape, and mass margins data. 
3. Categories that have different spelling, but the same meaning. 

In addition, processing and analyzing calcification and mass case datasets separately doesn't support a comprehensive, top-down view of the data. The following tasks will eliminate redundancy and produce a clean dataset for downstream analyses.

| # | Task                                                                             |
|---|----------------------------------------------------------------------------------|
| 1 | Create a combined calcification and mass case dataset for analysis.              |
| 2 | Replace invalid values for breast density with NA for imputation.                |
| 3 | Set mass and calcification morphological values to NA where they do not   apply. |
| 4 | Remove cases with invalid values for subtlety                                    |
| 5 | Replace calcification types 'LUCENT_CENTER' with 'LUCENT_CENTERED', and 'PLEOMORPHIC-PLEOMORPHIC', with 'PLEOMORPHIC' |
| 6 | Impute missing values using Multiple Imputation by Chained Equations   (MICE)    |
| 7 | Convert data to the appropriate data types                                       |
| 8 | Conduct random inspection of imputations.                                        |
| 9 | Conduct final data quality analysis.                                             |

Once the case dataset has been cleaned, we will merge it with the DICOM dataset to create the DICOM-CASE dataset.

In [9]:
import os
if 'jbook' in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))

import pandas as pd
import numpy as np

from bcd.data.prep.meta.case import CaseImputer
pd.options.display.max_rows = 999

from bcd.data.quality.case import MassCaseDQA, CalcCaseDQA

In [10]:
FP_STAGED_CALC = "data/staged/calc_cases.csv"
FP_STAGED_MASS = "data/staged/mass_cases.csv"
FP_STAGED = "data/staged/cases.csv"

FP_CLEAN_CASES = "data/clean/cases.csv"
FP_CLEAN_CALC =  "data/clean/calc_cases.csv"
FP_CLEAN_MASS =  "data/clean/mass_cases.csv"
CASES_TO_REMOVE = ['P_00710_RIGHT_mass_MLO_1', 'P_00710_RIGHT_mass_CC_1']

## Create Combined Case Dataset

In [11]:
dfc = pd.read_csv(FP_STAGED_CALC)
dfm = pd.read_csv(FP_STAGED_MASS)
df = pd.concat([dfc,dfm], axis=0)
df_orig = df.copy()


## Breast Density
Replace invalid values for breast density with NA for downstream imputation.

In [12]:
# Set invalid values for breast_density to NA
df['breast_density'] = df['breast_density'].replace(0, np.NAN)

## Morphology Not Applicable
Set morphological variables to NA where they do not apply. For instance mass shape doesn't apply to calcification cases.

In [13]:
# Set case and mass data to NOT APPLICABLE where appropriate.
df.loc[df['abnormality_type'] == 'calcification', 'mass_shape'] = 'NOT APPLICABLE'
df.loc[df['abnormality_type'] == 'calcification', 'mass_margins'] = 'NOT APPLICABLE'
df.loc[df['abnormality_type'] == 'mass', 'calc_type'] = 'NOT APPLICABLE'
df.loc[df['abnormality_type'] == 'mass', 'calc_distribution'] = 'NOT APPLICABLE'

## Case Removal
Remove cases with invalid values for subtlety.

In [15]:
df = df.loc[~df['case_id'].isin(CASES_TO_REMOVE)]
msg = f"A total of {df_orig.shape[0] - df.shape[0]} rows were removed from the dataset, leaving {df.shape[0]} rows."
print(msg)

KeyError: 0

## Category Alignment
Align categories that have the same meaning, but differ in spelling only.

In [7]:
df.loc[df['calc_type'] == 'LUCENT_CENTER', 'calc_type'] = 'LUCENT_CENTERED'
df.loc[df['calc_type'] == 'ROUND_AND_REGULAR-LUCENT_CENTER-DYSTROPHIC', 'calc_type'] = 'ROUND_AND_REGULAR-LUCENT_CENTERED-DYSTROPHIC'
df.loc[df['calc_type'] == 'PUNCTATE-LUCENT_CENTER', 'calc_type'] = 'PUNCTATE-LUCENT_CENTERED'
df.loc[df['calc_type'] == 'VASCULAR-COARSE-LUCENT_CENTER-ROUND_AND_REGULAR-PUNCTATE', 'calc_type'] = 'VASCULAR-COARSE-LUCENT_CENTERED-ROUND_AND_REGULAR-PUNCTATE'
df.loc[df['calc_type'] == 'ROUND_AND_REGULAR-LUCENT_CENTER', 'calc_type'] = 'ROUND_AND_REGULAR-LUCENT_CENTERED'
df.loc[df['calc_type'] == 'LUCENT_CENTER-PUNCTATE', 'calc_type'] = 'LUCENT_CENTERED-PUNCTATE'
df.loc[df['calc_type'] == 'COARSE-ROUND_AND_REGULAR-LUCENT_CENTER', 'calc_type'] = 'COARSE-ROUND_AND_REGULAR-LUCENT_CENTERED'
df.loc[df['calc_type'] == 'ROUND_AND_REGULAR-LUCENT_CENTER-PUNCTATE', 'calc_type'] = 'ROUND_AND_REGULAR-LUCENT_CENTERED-PUNCTATE'
df.loc[df['calc_type'] == 'COARSE-LUCENT_CENTER', 'calc_type'] = 'COARSE-LUCENT_CENTERED'
df.loc[df['calc_type'] == 'PLEOMORPHIC-PLEOMORPHIC', 'calc_type'] = 'PLEOMORPHIC'

## Impute Missing Values
Multiple Imputation by Chained Equations (MICE) is a robust, informative method of estimating missing values in datasets. The procedure imputes missing data through an iterative series of predictive models which estimate the value of missing data using the other variables in the dataset. For this, we'll use our CaseImputer which wraps scikit-learn's IterativeImputer implementation of MICE.

First, let's capture the missing values as we will inspect them after imputation.

In [8]:
# Grab rows with missing data
null_mask = df.isnull().any(axis=1)
df_missing = df[null_mask]
msg = f"There are {df_missing.shape[0]} rows (approximately {round(df_missing.shape[0] / df_orig.shape[0] * 100,1)}% of the dataset) with missing data in the dataset."
print(msg)

NameError: name 'df_orig' is not defined

In [ ]:
imp = CaseImputer(random_state=5)
imp.fit(df=df)
df_clean = imp.transform(df=df)

With that, let's save the data.

## Data Types

In [ ]:
dtypes = {"patient_id": "object",
"breast_density": "int32",
"left_or_right_breast": "category",
"image_view": "category",
"abnormality_id": "int32",
"abnormality_type": "category",
"calc_type": "category",
"calc_distribution": "category",
"assessment": "int32",
"pathology": "category",
"subtlety": "int32",
"fileset": "object",
"mass_shape": "category",
"mass_margins": "category",
"case_id": "object",
"cancer": "bool",
}
dtypes_ordinal = {
"breast_density": "category",
"assessment": "category",
"subtlety": "category",
}
df_clean = df_clean.astype(dtypes)
df_clean = df_clean.astype(dtypes_ordinal)
df_clean.info()
df_clean.head()

In [ ]:
os.makedirs(os.path.dirname(FP_CLEAN_CASES), exist_ok=True)
df_clean.to_csv(FP_CLEAN_CASES, index=False)

## Random Sample Inspection

Let's take a look at a random sampling of the missing data and compare.

In [ ]:
sample_cases = df_missing['case_id'].sample(10)
df_missing.loc[df_missing['case_id'].isin(sample_cases)]
df_clean.loc[df_clean['case_id'].isin(sample_cases)]

## Data Quality Analysis 2.0
Ok, let's have another go at the data quality analysis.

### Completeness Analysis

In [ ]:
dqc = CalcCaseDQA(filepath=FP_CLEAN_CALC)
dqm = MassCaseDQA(filepath=FP_CLEAN_MASS)
dqcc = dqc.analyze_completeness()
dqmc = dqm.analyze_completeness()

In [ ]:
print(dqcc.summary)
print(dqmc.summary)

We're complete.

### Uniqueness Analysis

In [ ]:
dqcu = dqc.analyze_uniqueness()
dqmu = dqm.analyze_uniqueness()
print(dqcu.summary)
print(dqmu.summary)

We're unique.

### Validity Analysis

In [ ]:
dqcv = dqc.analyze_validity()
dqmv = dqm.analyze_validity()
print(dqcv.summary)
print(dqmv.summary)

We're valid. This concludes the data cleaning exercise.